In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import correlate
from skimage.metrics import structural_similarity as ssim
import os

# Load the .wav files (natural signal and composite signal)
folder_path = r'/mnt/external_disk/InsectSound1000'
insect_name = 'Aphidoletes_'
audio_data_dict = {}

# Load audio files
wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav') and insect_name in f]
for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    try:
        audio_data, sample_rate = librosa.load(file_path, sr=None)
        audio_data_dict[wav_file] = {'audio': audio_data, 'sample_rate': sample_rate}
    except Exception as e:
        print(f"Error loading {wav_file}: {e}")
        
random_wav_file = wav_files[0]
natural_signal = audio_data_dict[random_wav_file]['audio']

composite_signal, sr_composite = librosa.load('Signals/Aphidoletes_composite_signal.wav', sr=None)

# Ensure both signals have the same sample rate
if sample_rate != sr_composite:
    raise ValueError("Sample rates do not match between the two signals.")

# 1. Cross-Correlation to measure overlap
def cross_correlation(signal1, signal2):
    # Normalize the signals
    signal1 = (signal1 - np.mean(signal1)) / np.std(signal1)
    signal2 = (signal2 - np.mean(signal2)) / np.std(signal2)
    # Calculate cross-correlation
    corr = correlate(signal1, signal2, mode='full')
    return corr

cross_corr = cross_correlation(natural_signal, composite_signal)
lags = np.arange(-len(natural_signal) + 1, len(natural_signal))

# Plot the cross-correlation
plt.figure(figsize=(10, 6))
plt.plot(lags, cross_corr)
plt.title('Cross-Correlation between Natural and Composite Signals')
plt.xlabel('Lag')
plt.ylabel('Correlation')
plt.show()

# 2. Spectrogram Comparison
def plot_spectrogram(signal, sr, title, max_freq=500):
    plt.figure(figsize=(10, 6))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(signal)), ref=np.max)
    librosa.display.specshow(D, x_axis='time', y_axis='linear', sr=sr)
    plt.ylim(0, max_freq)  # Limit the frequency axis to 0 to max_freq Hz
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.show()

# Plot spectrograms with frequency capped at 500 Hz
plot_spectrogram(natural_signal, sample_rate, 'Natural Signal Spectrogram (0-500 Hz)', max_freq=500)
plot_spectrogram(composite_signal, sr_composite, 'Composite Signal Spectrogram (0-500 Hz)', max_freq=500)


# 3. Signal-to-Interference Ratio (SIR)
def signal_to_interference_ratio(natural, composite):
    # Calculate the energy of the natural signal
    natural_energy = np.sum(natural ** 2)
    # Calculate the energy of the composite signal
    interference_energy = np.sum(composite ** 2)
    # Calculate SIR (ratio of signal to interference energy)
    sir = 10 * np.log10(natural_energy / interference_energy)
    return sir

sir_value = signal_to_interference_ratio(natural_signal, composite_signal)
print(f'Signal-to-Interference Ratio (SIR): {sir_value:.2f} dB')

# 4. SSIM (Structural Similarity Index)
def calculate_ssim(signal1, signal2):
    # Rescale signals to the range [0, 255] for SSIM
    signal1_rescaled = np.uint8((signal1 - np.min(signal1)) / (np.max(signal1) - np.min(signal1)) * 255)
    signal2_rescaled = np.uint8((signal2 - np.min(signal2)) / (np.max(signal2) - np.min(signal2)) * 255)
    
    # Calculate SSIM between the two signals
    return ssim(signal1_rescaled, signal2_rescaled)

ssim_value = calculate_ssim(natural_signal, composite_signal)
print(f'Structural Similarity Index (SSIM): {ssim_value:.4f}')

# Signal-to-Noise Ratio (SNR)
def calculate_snr(natural, composite):
    # Calculate the signal power
    signal_power = np.mean(natural ** 2)
    # Calculate the noise power (difference between signals)
    noise_power = np.mean((natural - composite) ** 2)
    # Calculate SNR
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

snr_value = calculate_snr(natural_signal, composite_signal)
print(f'Signal-to-Noise Ratio (SNR): {snr_value:.2f} dB')


# Time arrays for plotting
time_natural = np.linspace(0, len(natural_signal) / sample_rate, len(natural_signal))
time_composite = np.linspace(0, len(composite_signal) / sr_composite, len(composite_signal))

# Plot signals in the time domain
plt.figure(figsize=(12, 8))

# Natural signal
plt.subplot(2, 1, 1)
plt.plot(time_natural, natural_signal, label='Natural Signal', color='blue')
plt.title('Natural Signal (Time Domain)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()
plt.legend()

# Composite signal
plt.subplot(2, 1, 2)
plt.plot(time_composite, composite_signal, label='Composite Signal', color='orange')
plt.title('Composite Signal (Time Domain)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()